In [45]:
import pandas as pd

In [46]:
df = pd.DataFrame()  # data frame vazio a ser populado

In [47]:
def read_files(path, name_file, year_date, type_file):
    #path to file
    _file = f'{path}{name_file}{year_date}.{type_file}'
    # lista com os campos que queremos pegar
    colspecs = [
        (2,10),
        (10,12),
        (12,24),
        (27,39),
        (56,69),
        (69,82),
        (82,95),
        (108,121),
        (152,170),
        (170,188)
        ]
    # lista de nomes dos campos que estamos criando/pegando
    names = ['data_pregao', 'codbdi', 'sigla_acao', 'nomes_acao', 'preco_abertura', 'preco_maximo',        'preco_minimo', 'preco_fechamento', 'qtd_negocios', 'volume_negocios' ]

    df = pd.read_fwf(_file, colspecs=colspecs, names=names, skiprows=1)  # skip primeira linha

    return df

In [48]:
df.head()  # vemos que temos campos que não estão bem formatados (como a data, por exemplo)

""


In [49]:
# filtrar ações
def filter_stocks(df):
    df = df[df['codbdi']==2]
    df = df.drop(['codbdi'], 1)  # não precisamos mais do campo codbdi, na coluna 1
    
    return df

In [50]:
# ajuste do campo de data (formatação)
def parse_date(df):
    df['data_pregao'] = pd.to_datetime(df['data_pregao'], format='%Y%m%d')
    
    return df

In [51]:
# ajuste dos campos numéricos ('/100' para duas casas decimais e para 'float')
def parse_values(df):
    df['preco_abertura'] = (df['preco_abertura']/100).astype(float)
    df['preco_maximo'] = (df['preco_maximo']/100).astype(float)
    df['preco_minimo'] = (df['preco_minimo']/100).astype(float)
    df['preco_fechamento'] = (df['preco_fechamento']/100).astype(float)
    df['qtd_negocios'] = (df['qtd_negocios']).astype(int) #conversão para int
    df['volume_negocios'] = (df['volume_negocios']).astype(int) #conversão para int
    
    return df

In [52]:
# juntando os arquivos

def concat_files(path, name_file, year_date, type_file, final_file):
    # enumerate pega a lista e coloca o indice e o campo da informação (i e y, respectivamente)
    for i, y in enumerate(year_date):
        df = read_files(path, name_file, y, type_file)
        df = filter_stocks(df)
        df = parse_date(df)
        df = parse_values(df)

        if i==0:
            df_final = df
        else:
            df_final = pd.concat([df_final, df])
    
    df_final.to_csv(f'{path}/{final_file}', index=False)

In [53]:
#executando o programa de ETL ('Extract, Transform, and Load')

year_date = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021']

path = f'./cotacoes/'

name_file='COTAHIST_A'

type_file = 'txt'

final_file = 'all_bovespa.csv'

concat_files(path, name_file, year_date, type_file, final_file)